In [1]:
import os
import sys

from collections import defaultdict, Counter
from tqdm.notebook import tqdm_notebook

import numpy as np
import pandas as pd
import scipy.stats as sps
import scipy.sparse as scsp
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import ndcg_score, dcg_score, roc_auc_score, average_precision_score
from sklearn.metrics.pairwise import cosine_similarity

from joblib import Parallel, delayed

import tqdm
import json

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, clear_output

sns.set()

# Dataset

In [4]:
!wget -O zen_dataset.tar.gz https://www.dropbox.com/s/15mkthbvlturbsz/zen_dataset.tar.gz?dl=0
!tar -xzvf zen_dataset.tar.gz

zsh:1: no matches found: https://www.dropbox.com/s/15mkthbvlturbsz/zen_dataset.tar.gz?dl=0
tar: Error opening archive: Failed to open 'zen_dataset.tar.gz'


In [5]:
!tar -xzvf zen_dataset.tar.gz

x zen_dataset/
x zen_dataset/user_ratings.gz
x zen_dataset/.ipynb_checkpoints/
x zen_dataset/item_counts.csv
x zen_dataset/item_meta.gz


In [6]:
item_counts = pd.read_csv('zen_dataset/item_counts.csv', index_col=0)
item_meta = pd.read_csv('zen_dataset/item_meta.gz', compression='gzip', index_col=0)
user_ratings = pd.read_csv('zen_dataset/user_ratings.gz', compression='gzip', index_col=0)

In [7]:
item_counts['itemId'] = item_counts['itemId'].apply(str)

Dataset consists of text articles from dzen.ru and user interactions on them.
* 105000 items
* 95000 users
* 40 mln interactions

In [8]:
with pd.option_context('display.max_colwidth', 100):
    display(item_meta)

,itemId,title,content
0,5480844460835530524,"Нехитрые способы, как самостоятельно проверить качество воды из скважины или колодца","С раннего детства нам рассказывают, что самая вкусная и полезная вода – та, что добывается из ..."
1,25708764690236829,"Где находилась сверхсекретная база подводных лодок СССР, которую совершенно не видно с моря?","Сомневаюсь, что найдётся сейчас человек, который никогда не слышал о Балаклаве. Даже если допуст..."
2,25995859650472943,Тапки ( жуткий рассказ),"Год назад эту историю рассказала мне моя родственница и с её позволения, я хочу ей поделиться...."
3,26039067597386753,Крутые находки на Aliexpress №1113,"Доброго времени суток, Уважаемые читатели! Добро пожаловать на канал ""Чайна Таун"", где ежедневно..."
4,26225874317634871,Нам пообещали высокую инфляцию. Деньги сильно обесценятся — и целое поколение будет «жить в долгах»,"Цены в магазинах и темпы инфляции растут не только в нашей стране, но и практически во всех стра..."
...,...,...,...
104498,6221825086402198588,Сколько можно заработать на видеокартах с AGP интерфейсом,"Очень старые видеокарты с интерфейсом AGP, которые как кажется, никому уже не нужны. К нам в маг..."
104499,6221897338759013055,Укрытие Роз на зиму,По Вашим просьбам - повторяю пост прошлого года - про укрытие Роз ( с небольшой редакцией)\n\nЯ ...
104500,6221960724554910431,"Мама, мамочка, мамуля: снимки самого близкого человека на свете",В подборке снимков из нашего архива — фотографии мам со своими детьми времен царской России и СС...
104501,6222047264920702976,"Что лучше: сдавать наследственную квартиру или продать? Рассказываю, почему я выбрал второй вариант","""Кварта − новый бренд ПИК-Брокер"" Привет, меня зовут Андрей, мне 30 лет. Я вообще никогда не пис..."


In [9]:
user_ratings

,userId,trainRatings,testRatings
0,-993675863667353526,"{'-5222866277752422391': 0, '-9060464686933784...","{'-2554466548053893601': 0, '-2220576615613681..."
1,-4250619547882954185,"{'-4553947455665416667': 0, '-3876917199970727...","{'-3523829386334236920': 0, '-1723368694207177..."
2,-3847785305345691076,"{'1455441194337562599': 0, '-17536433251064509...","{'1148168316930968740': 0, '706941777097091385..."
3,1785181112918558233,"{'-2853304815794005643': 0, '-8508657381620121...","{'4719491585277936855': 1, '706941777097091385..."
4,-5078748097863903181,"{'-742528302744844176': 1, '-82389094081541106...","{'6294770073358764390': 0, '-40435255593741244..."
...,...,...,...
75905,4954138831959898373,"{'-6764562552262937310': 0, '-7209088773475894...","{'-1685266709911581206': 0, '-5318736529174157..."
75906,4967793435819938014,"{'6602646631687202064': 0, '-38704649420060130...","{'6266145763213971476': 0, '-37217722565891246..."
75907,-7137764184903122777,"{'-7841343107825813783': 0, '-6743959661482653...","{'482864988385991583': 1, '-132806041087172535..."
75908,-2624987805086334956,"{'-8048759438586949657': 0, '-7609637705975070...","{'-5614425467424704155': 0, '62661457632139714..."


In [12]:
user_encoder = LabelEncoder().fit(user_ratings['userId'])
item_encoder = LabelEncoder().fit(item_counts['itemId'])

In [13]:
all_items = item_counts['itemId']
indices = item_encoder.transform(all_items)
item_to_id = dict(zip(all_items, indices))

# SLIM

Let be $R \in \mathbb{R}^{|U| \times |I|}$ &mdash; binary matrix of interactions between users and items. Then SLIM optimizes

$$L = \frac{1}{2} \Vert R - RW \Vert_F^2 + \frac{\beta}{2} \Vert W \Vert_F^2 + \lambda \Vert W \Vert_1 \rightarrow \min_W,\\
  s.t. \forall i, j \; W_{ij} \geq 0, W_{ii} = 0.$$
  
$W$ is a matrix of pairwise item-item similarities.

$$
\frac{1}{2} \Vert R - RW \Vert_2^2 + \frac{\beta}{2} \Vert W \Vert_2^2 + \lambda \Vert W \Vert_1
$$

Consider item $i$

$$
\frac{1}{2} \sum_j \Vert r_j - Rw_j \Vert_2^2 + \lambda \Vert w_j \Vert_1 + \frac{\beta}{2} \Vert w_j \Vert_2^2
$$

We use numba library to implement fast operations in Python.

https://numba.pydata.org/

Also recently `torch` become able to compile operations in the same fashion:\
https://pytorch.org/blog/compiling-numpy-code/

Another option to speedup your code: https://cython.org/

One more way to speedup is https://docs.cupy.dev/en/stable/index.html (with GPU)

Before function definition we need to write: `@numba.njit()`.

In [16]:
import numba

item_ratings_ind = [numba.typed.List() for _ in range(len(item_encoder.classes_))]
user_ids = user_encoder.transform(user_ratings['userId'])

for user_id, items_with_ratings in tqdm_notebook(zip(user_ids, user_ratings['trainRatings']),
                                                 total=len(user_ratings)):
    item_ids, item_ratings = zip(*json.loads(items_with_ratings.replace("'", '"')).items())
    item_ids = [item_to_id[item_id] for item_id in item_ids]
    for item_id, rating in zip(item_ids, item_ratings):
        item_ratings_ind[item_id].append((user_id, rating))

  0%|          | 0/75910 [00:00<?, ?it/s]

In [17]:
item_ratings_ind_nb = numba.typed.List(item_ratings_ind)

$$
w_{jk} = \frac{r^T_j r_k - \lambda - \sum_{i \ne k} w_{ji} r^T_i r_k}{r^T_k r_k + \beta}
$$

In [21]:
@numba.njit()
def fit_one_item(item_ratings, j, n_iter=20, l2_reg=1.0, l1_reg=2.0):
    """Optimizes one column of W matrix
    * item_ratings -- list of user-item interactions список списков взаимодействий айтема,
      item_ratings[j] -- list of interactions of all users with item j
    * j -- item for which we search for weights
    * n_iter -- number of iterations
    * l1_reg, l2_reg -- regularization coefficients

    Retruns:
      dict: item -> weight, for all non zero weights
    """
    n_items = len(item_ratings)
    per_item_positives = []
    item_interactions = set()

    for user, rating in item_ratings[j]:
        item_interactions.add(user)

    for i in range(n_items):
        positives = set() # users
        for user, rating in item_ratings[i]:
            positives.add(user)
        per_item_positives.append(positives)

    w = np.zeros(n_items)
    non_zero_items = set()
    for _ in range(n_iter):
        for k in range(n_items):
            if k == j:
                continue

            score = len(per_item_positives[j] & per_item_positives[k]) - l1_reg
            for i in non_zero_items:
                if i == k:
                    continue

                score -= w[i] * len(per_item_positives[i] & per_item_positives[k])

                if score < 0:
                    break
            score /= len(per_item_positives[k]) + l2_reg
            
            score = max(score, 0.0)

            w[k] = score
            if w[k] > 1e-5:
                non_zero_items.add(k)

    non_zero_elements = {}
    for i, value in enumerate(w):
        assert value >= 0.0
        if value > 0:
            non_zero_elements[i] = value

    return non_zero_elements

In [22]:
def get_item_meta(item_id):
    item_id = int(item_encoder.inverse_transform([item_id])[0])
    return item_meta[item_meta['itemId'] == item_id].iloc[0].to_dict()


def visualize_top(item_ratings_ind_nb, j, top=10):
    weights = fit_one_item(item_ratings_ind_nb, j)
    sorted_items = sorted(weights.items(), key=lambda x: x[1], reverse=True)[:top]

    item_ids, weights = zip(*sorted_items)
    items = map(get_item_meta, item_ids)
    anchor_item = get_item_meta(j)

    with pd.option_context('display.max_colwidth', 100):
        display(pd.DataFrame({
            anchor_item['title']: [item['title'] for item in items],
            'score': weights
        }))

In [23]:
visualize_top(item_ratings_ind_nb, 1)

,"7 новых комиксов от проекта «Мозги трески» - Волос в супе, послание в бутылке и другие истории",score
0,"«Пикник монстров» - Новая серия комиксов о забавных разноцветных человечках, которые живут в мир...",0.121191
1,"11 правдивых комиксов о привычках парней, которые мало кто знает",0.076983
2,Ирландский художник уже больше 7 лет рисует комиксы почти каждый день и его работы полны черного...,0.063941
3,Чья эволюция шагнула дальше: людей или котиков?,0.063211
4,"Костюмы, нечисть и конфеты - 10 комиксов про Хэллоуин от зарубежных авторов",0.059948
5,"Художник из Ливерпуля рисует смешные комиксы, вдохновляясь своей забавной семьей и друзьями",0.055897
6,И ещё одна подборка комиксов по сказкам,0.053048
7,"Художник из Чикаго рисует комиксы об оптимистичном кролике, юмор в которых понятен без слов",0.049730
8,"Художник из Техаса рисует саркастичные комиксы о современном обществе, а также высмеивает кинема...",0.044145
9,Художница стикеров из Волгограда рисует жизненные комиксы о работе на фрилансе и проблемах взрос...,0.043299


In [ ]:
visualize_top(item_ratings_ind_nb, 50)

,«Роскошная кухня после ремонта!» как выглядит квартира в Риге Татьяны Веденеевой - ФОТО,score
0,"Личная жизнь, карьера, внешность Таисии Повалий",0.172762
1,Убирайте из своих образов устаревшие стереотипы и после 60 вы будете молодой и стильной. А кому ...,0.145176
2,"Породистых леди видно сразу: 5 мелочей, которые выдают в женщине дешёвку!",0.140144
3,"4 российские актрисы, которые в 60-летнем возрасте выглядят на все 100",0.138545
4,Зрительница требует выгнать Милохина и почему Даниил Глейхенгауз желает победы Медведевой в шоу ...,0.113499
5,"Нет ничего страшнее того «поезда», под который она попала",0.111025
6,Прекрасная мама: Российские знаменитости и звезды шоу-бизнеса опубликовали трогательные и нежные...,0.104713
7,Что стало с бывшей женой-миллионершей и брошенным сыном Влада Сташевского,0.102414
8,"Любимая «мелодия» Муслима Магомаева, Тамара Синявская – в печали и в блеске драгоценностей после...",0.100501
9,"3 причины, почему я перестала звать гостей к себе домой, и дело не в экономии",0.100398


In [ ]:
visualize_top(item_ratings_ind_nb, 100)

,Забияко после вылета раскрыла все карты и рассказала чем осталась недовольна на шоу Ледниковый период 2021,score
0,"3 гениальные еврейские пословицы, которые спасают от депрессии и хандры",0.243903
1,«Какое стильное пальтишко у Аллы Пугачёвой!» - редкие ФОТО с праздника примадонны,0.231321
2,Халтура с нарядами Алины Загитовой и по чьей протекции попал на Ледниковый период Федор Федотов,0.202943
3,"Три мировых рекорда и российский пьедестал. Что скажете, госпожа Дюамель?",0.148041
4,"Ледниковый период 8 выпуск, последняя номинация, лучшие пары, оценки дуэтов, рассказываем подроб...",0.146016
5,Загадка друзей Валентины Малявиной,0.137659
6,В день рождения он спешит на лед: Алексею Тихонову - 50. Тернистый путь в спорте и семья. Чем се...,0.132900
7,Что обычно ожидало русских казачек в руках у кавказских горцев. Рассказываем,0.128706
8,Костя Богомолов на сцене театра,0.128252
9,"Диана Анкудинова получила трогательное послание от Стаса Костюшкина. Тёплые отголоски ""Шоумаскгоон""",0.122691
